# Texture Features

In [1]:
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
import pandas as pd
import time
import itertools
from math import sqrt, log, atan, degrees

from skimage import feature
from skimage.transform import resize

### Load data

In [2]:
df = pd.read_csv('../data/ATR_GT_Training.csv', header=None, names=['file','label'])
df.file = df.file.map(lambda x: x.replace("'",''))
df['img'] = df.file.map(lambda x: nib.load('../data/' + str(x) + '.nii.gz'))

### Define function to quantize and image

In [3]:
def quantize_image(data, n_levels):
    steps = np.max(data) / n_levels
    if steps == 0: steps = 1
    return (data / steps).astype(int)

### Define functions to compute GLCM and related features
Full feature list includes: GLCV, angular_2nd_moment, energy, contrast, dissimilarity, entropy, mean, var, std_dev, correlation, and similarity

In [4]:
# https://prism.ucalgary.ca/handle/1880/51900 page 11
# I = quantized img, Ng = number of gray levels
def compute_glcm(I, Ng, dx, dy, dz, symmetric=True):
    def get_range(Ng, d):
        if d >= 0:
            return range(Ng-d)
        else:
            return range(-d,Ng)
    
    glcm = np.zeros((Ng,Ng))
    range_x = get_range(I.shape[0], dx)
    range_y = get_range(I.shape[1], dy)
    range_z = get_range(I.shape[2], dz)
    
    ix = itertools.product(range(Ng), range(Ng), range_x, range_y, range_y)
    for i, j, x, y, z in ix:
        if I[x,y,z] == i and I[x+dx, y+dy, z+dz] == j:
            glcm[i,j] += 1
    if symmetric:
        glcm += glcm.T
    return glcm / np.sum(glcm)

# https://prism.ucalgary.ca/handle/1880/51900 page 24
def compute_glcv(glcm, Ng):
    glcv = np.zeros((Ng))
    for i, j in itertools.product(range(Ng), range(Ng)):
        glcv[abs(i-j)] += glcm[i,j]
    return glcv

# https://prism.ucalgary.ca/handle/1880/51900 page 30
def compute_contrast(glcm, Ng):
    contrast = 0
    for i, j in itertools.product(range(Ng),range(Ng)):
        contrast += (i-j)**2 * glcm[i,j]
    return contrast

# https://prism.ucalgary.ca/handle/1880/51900 page 32
def compute_dissimilarity(glcm, Ng):
    dissimilarity = 0
    for i, j in itertools.product(range(Ng),range(Ng)):
        dissimilarity += abs(i-j) * glcm[i,j]
    return dissimilarity

# https://prism.ucalgary.ca/handle/1880/51900 page 32
def compute_homogeneity(glcm, Ng):
    homogeneity = 0
    for i, j in itertools.product(range(Ng),range(Ng)):
        homogeneity += glcm[i,j] / (1 + (i-j)**2)
    return homogeneity
        
# https://prism.ucalgary.ca/handle/1880/51900 page 36
def compute_angular_2nd_moment(glcm, Ng):
    a2m = 0
    for i, j in itertools.product(range(Ng),range(Ng)):
        a2m += glcm[i,j]**2
    return a2m

# https://prism.ucalgary.ca/handle/1880/51900 page 36
def compute_energy(glcm, Ng):
    return sqrt(compute_angular_2nd_moment)

# https://prism.ucalgary.ca/handle/1880/51900 page 37
def compute_entropy(glcm, Ng):
    entropy = 0
    for i, j in itertools.product(range(Ng),range(Ng)):
        if glcm[i,j] != 0:
            entropy += glcm[i,j] * -log(glcm[i,j])
    return entropy

# https://prism.ucalgary.ca/handle/1880/51900 page 40
def compute_glcm_mean(glcm, Ng, axis):
    mean = 0
    for i, j in itertools.product(range(Ng),range(Ng)):
        if axis == 0:
            mean += i * glcm[i,j]
        elif axis == 1:
            mean += j * glcm[i,j]
    return mean

# https://prism.ucalgary.ca/handle/1880/51900 page 41
def compute_glcm_variance(glcm, Ng, axis, mean=None):
    if mean == None:
        mean = compute_glcm_mean(glcm, Ng, axis)
        
    variance = 0
    for i, j in itertools.product(range(Ng),range(Ng)):
        if axis == 0:
            variance += (i-mean)**2 * glcm[i,j]
        elif axis == 1:
            variance += (j-mean)**2 * glcm[i,j]
    return variance

# https://prism.ucalgary.ca/handle/1880/51900 page 41
def compute_glcm_std_dev(glcm, Ng, axis, mean=None):
    return sqrt(compute_glcm_variance(glcm, Ng, axis))

# https://prism.ucalgary.ca/handle/1880/51900 page 43
def compute_correlation(glcm, Ng, u_i, u_j, var_i, var_j):
    correlation = 0
    for i, j in itertools.product(range(Ng),range(Ng)):
        correlation += glcm[i,j] * (i - u_i) * (j - u_j) / sqrt(var_i * var_j)
    return correlation

# https://prism.ucalgary.ca/handle/1880/51900 page 43
def compute_similarity(glcm, Ng):
    similarity = 0
    for i, j in itertools.product(range(Ng),range(Ng)):
        similarity += glcm[i,j] / (1 + abs(i-j))
    return similarity

### Compute texture features for all images

In [5]:
Ng = 8
displacement_vectors = [[0,1,0],[-1,1,0],[-1,0,0],[-1,-1,0],
                       [0,1,-1],[0,0,-1],[0,-1,-1],[-1,0,-1],
                       [1,0,-1],[-1,1,-1],[1,-1,-1],[-1,-1,-1],[1,1,-1]]
texture_flat = []

for i, img in enumerate(df.img):
    data = img.get_fdata()
    data32 = resize(data, (16,16,16), mode='reflect', anti_aliasing=False)
    quantized_img = quantize_image(data32, Ng)
    
    texture_features = []
    for vector in displacement_vectors[:]:
        dx,dy,dz = vector
        start = time.time()
        glcm = compute_glcm(quantized_img, Ng, dx, dy, dz)
        glcv = compute_glcv(glcm, Ng)
        angular_2nd_moment = compute_angular_2nd_moment(glcm, Ng)
        energy = sqrt(angular_2nd_moment)
        contrast = compute_contrast(glcm, Ng)
        dissimilarity = compute_dissimilarity(glcm, Ng)
        entropy = compute_entropy(glcm, Ng)
        mean = compute_glcm_mean(glcm, Ng, 0)
        var = compute_glcm_variance(glcm, Ng, 0, mean)
        std_dev = sqrt(var)
        correlation = compute_correlation(glcm, Ng, mean, mean, var, var)
        similarity = compute_similarity(glcm, Ng)
        features = list(glcv) + [angular_2nd_moment, energy, contrast, dissimilarity,
                               entropy, mean, var, std_dev, correlation, similarity]
        texture_features.append(features)
    texture_features = np.array(texture_features)
    texture_flat.append(list(np.resize(texture_features, np.product(texture_features.shape))))
    if i % 25 == 0 and i != 0:
        print('Finished ' + str(i) + ' images.')
print('Finished')

Finished 25 images.


C:\Users\coone\AppData\Local\Temp\ipykernel_4972\3418793545.py:101: RuntimeWarning: invalid value encountered in scalar divide
  correlation += glcm[i,j] * (i - u_i) * (j - u_j) / sqrt(var_i * var_j)


Finished 50 images.
Finished 75 images.
Finished 100 images.
Finished 125 images.
Finished 150 images.
Finished 175 images.
Finished 200 images.
Finished 225 images.
Finished 250 images.
Finished 275 images.
Finished 300 images.
Finished 325 images.
Finished 350 images.
Finished 375 images.
Finished 400 images.
Finished 425 images.
Finished 450 images.
Finished 475 images.
Finished 500 images.
Finished 525 images.
Finished 550 images.
Finished 575 images.
Finished 600 images.
Finished 625 images.
Finished 650 images.
Finished 675 images.
Finished 700 images.
Finished 725 images.
Finished 750 images.
Finished 775 images.
Finished 800 images.
Finished 825 images.
Finished 850 images.
Finished 875 images.
Finished 900 images.
Finished 925 images.
Finished 950 images.
Finished 975 images.
Finished 1000 images.
Finished 1025 images.
Finished 1050 images.
Finished 1075 images.
Finished 1100 images.
Finished 1125 images.
Finished 1150 images.
Finished 1175 images.
Finished 1200 images.
Finish

In [6]:
np.save('features_texture.npy',np.array(texture_flat))